# Demystifying Neural Network
## Pytorch Tutorial 3

- Build a CNN model using Mnist dataset

### Intro
- In part 6, I learned how to build a neural network from scratch with Pytorch using this [tutorial](https://pytorch.org/tutorials/beginner/nn_tutorial.html). In this tutorial, I would like to go over the remaining part - building a CNN model.

### CNN model
- Pytorch의 `Conv2d` 클래스를 사용하여 convolution layer를 만들어 본다. 아래의 CNN모델은 이 `Conv2d` 의 3개의 convolution layer를 사용할 것이다. 각 convolution layer는 Rectified Linear Unit (ReLU) 함수를 통과 하고, 마지막으로는 average pooling을 거친다. 

**Average pooling이란?**
down sampling 기법 중 하나이다 (max pooling이 현재 많이 쓰이는 down sampling 기법이다). convolution layer 뒤에 붙는 "pooling" layer로 피쳐맵의 차원을 줄여주어 파라미터의 수를 줄여줌으로서 계산을 빠르게 해주는 장점이 있다. 그리하여 과최적화를 막아주어 모델의 성능을 향상시켜줄 수 있다. Average pooling operation은 아래의 그림을 보면 이해하기 쉽다. Pooling을 할 때 stride와 kernel size (filter size)를 지정해 주어야 하는데, overlapping, 즉 꼭 겹치지 않아야 하는 것은 아니다. AlexNet의 경우, [Kernel Size, Stride] = [3x3, 2] 이고, GoogleNet의 경우는 [3x3, 2] & [3x3, 1] 여서 겹치게 pooling을 한 CNN 구조가 꽤 있다.

<img src="img/blog7_figure1.png" width="600" height="300" />

- part 6에서 사용한 pytorch의 기능들을 그대로 사용한 후, CNN구조를 만들어 본다. 첫번째로, `pickle`을 사용하여 데이터를 불러온 후, 데이터를 tensor object로 바꾸어 준다

In [1]:
import pickle
import gzip
from pathlib import Path

DATA_PATH = Path("data")
PATH = DATA_PATH / "mnist"
FILENAME = "mnist.pkl.gz"

with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")

In [2]:
import torch 

x_train, y_train, x_valid, y_valid = map(torch.tensor, (x_train, y_train, x_valid, y_valid))

- `TensorDataset`과 `DataLoader`를 이용하여 train set과 validation set을 wrapping해준다. Validation set의 batch size는 part 6에서 설명한 것과 마찬가지로, validation set에는 back propagation을 할 필요가 없으니, batch size를 두배로 눌려주어 loss를 빠르게 계산한다

In [3]:
from torch.utils.data import TensorDataset, DataLoader

bs = 64
train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)

valid_ds = TensorDataset(x_valid, y_valid)
valid_dl = DataLoader(valid_ds, batch_size=bs * 2)

- CNN 구조를 만들어 보자. convolution layer 3개를 사용할 것이며, 각각에 layer에 ReLu nonlinearity를 입힌다. 
- 마지막으로는 average pooling을 사용하여 차원을 줄여준다.  

In [4]:
from torch import nn
import torch.nn.functional as F

loss_func = F.cross_entropy

class Mnist_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1)
        
    def forward(self, xb):
        xb = xb.view(-1, 1, 28, 28)
        xb = F.relu(self.conv1(xb))
        xb = F.relu(self.conv2(xb))
        xb = F.relu(self.conv3(xb))
        xb = F.avg_pool2d(xb, 4)
        return xb.view(-1, xb.size(1))

lr = 0.1

- part 6에서 사용한 batch의 loss를 구하는 `loss_batch`함수와 학습을 하는 `fit`함수를 그대로 사용하여 학습한다 

In [13]:
def get_data(train_ds, valid_ds, bs):
    return (
    DataLoader(train_ds, batch_size=bs, shuffle=True),
    DataLoader(valid_ds, batch_size=bs*2),
    )

In [5]:
import numpy as np

def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)
    
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
    
    return loss.item(), len(xb)
    
def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)
        
        model.eval()
        with torch.no_grad():
            losses, nums = zip(*[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl])
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)
        
        print(epoch, val_loss)

In [14]:
from torch import optim

epochs = 5

model = Mnist_CNN()
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 0.3722176811218262
1 0.27552606678009034
2 0.18493960452079772
3 0.1743204867362976
4 0.14157332420349122


### nn. Sequential
- `torch.nn`에는 `Sequential` 클래스가 있는데, `Sequential`을 사용하면 더 간단하게 neural network를 만들 수 있다. `Sequential`객체는 객체 안에 들어있는 모듈을 순서대로 실행한다. `Sequential`의 이점을 이용하기 위해서는 어떠한 함수가 주어졌을 때, 그 함수를 사용하여 쉽게 layer를 만들 수 있어야 한다. 예를들어 Pytorch에는 view (numpy의 reshape과 같다)를 적용해주는 layer가 없기 때문에 직접 만들어 주어야 한다. 아래의 `Lambda`클래스를 사용하면 `Sequential`클래스를 사용하여 neural network를 만들 때 layer를 생성할 수 있다.

In [11]:
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func
    
    def forward(self, x):
        return self.func(x)
    
def preprocess(x):
    return x.view(-1, 1, 28, 28)

In [12]:
model = nn.Sequential(
    Lambda(preprocess),
    nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.AvgPool2d(4),
    Lambda(lambda x: x.view(x.size(0), -1)),
)

opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
epochs=3
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 0.3974899738788605
1 0.2392974494934082
2 0.20098116474151612


### Wrapping DataLoader
- 위의 CNN 모델은 간단하지만, MNIST 데이터와만 작동이 된다. 그 이유는:  
1. input이 1 x 784 (28*28) 벡터임을 가정하고 있기 때문이고,
2. 마지막 CNN output 사이즈 또한 4*4 (average pooling kernel 사이즈) 임을 가정하고 있기 때문이다

위의 두 가정을 없애고 어떤 2d 이미지에서도 CNN모델이 작동될 수 있도록 만들어 보자. DataLoader를 wrapping하는 class를 만든다면 가능하다. 위에서 만들었던 lambda layer는 일단 생각하지 말고, 데이터 전처리를 generator로 만들어 준다

In [14]:
def preprocess(x, y):
    return x.view(-1, 1, 28, 28), y

class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func
    
    def __len__(self):
        return len(self.dl)
    
    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b))

train_dl, valid_dl = get_data(train_ds, valid_ds, bs)
train_dl = WrappedDataLoader(train_dl, preprocess)
valid_dl = WrappedDataLoader(valid_dl, preprocess)

이제 `nn.AvgPool2d`를 `nn.AdaptiveAvgPool2d`로 바꾸어 준다. `nn.AdaptiveAvgPool2d`는 `nn.AvgPool2d`와는 다르게 input 텐서 사이즈 지정이 아닌, output 텐서의 사이즈를 지정해 줄 수 있다. 그러므로 모델이 어떤 사이즈의 input이 들어와도 작동할 수 있게 해 준다.

In [15]:
model = nn.Sequential(
    nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    Lambda(lambda x: x.view(x.size(0), -1)),
)

In [16]:
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 0.399412823677063
1 0.27265121469497683
2 0.205451886844635


### Summary
- tutorial 2와 3에서 다루었던 모듈과 클레스인 `torch.nn`, `torch.optim`, `Dataset`, `DataLoader`에 대해 정리해보자
- **torch.nn**  
    - `Module`은 호출 가능한 함수를 만들고, "상태"를 가지고 있을 수 있다 (예를 들어, neural network layer의 weights). 또한 neural network가 어떤 parameter를 가지고 있는지 알고 있으며, 모든 gradient를 초기화 시키고, weight을 갱신할 수 있다
    - `Parameter`는 텐서를 wrapping한다. 그럼으로서 `Module`에게 지정된 텐서가 weights을 가지고 있으며 back propagation할 때 업데이트가 되어져야 한다는 것을 알려주는 역할을 한다. 기억해야 할 것은, `requires_grad`가 켜진 텐서들만 weights를 업데이트 한다는 것이다.
    - `functional`(주로 `F`라는 네임스페이스로 임포트 한다: `import torch.nn.functional as F`)은 사용자가 직접 actication function이나 loss function을 정의하지 않고 `functional`에서 이미 만들어진 함수를 사용할 수 있게 해주며, convolution layer와 linear layer또한 포함하고 있어 사용자가 쉽게 커스터마이징 하여 사용할 수 있다.
    - `torch.optim`는 다양한 최적화 방법을 가지고 있다. 위에서 사용했던 `SGD` (stochastic gradient descent) 클래스가 `torch.optim`모듈에 포함되어 있다.
    - `Dataset`은 사용자가 각 image에 접근할 수 있게 해주고, 가지고 이미지가 몇장인지를 알려준다. tutorial에서 `TensorDataset`을 사용하여 텐서 객체로 변환시킨 후, 각 이미지에 접근할 수 있었다
    - `DataLoader` - `Dataset`을 `DataLoader`로 만들어 주면, iterator로 만들어 주어 학습을 batch로 할 때 편하게 사용할 수 있다